# Twitter for Stock Prediction

In [ ]:
#Scrape twitter for tweets
#Clean tweets
#Analyse tweet sentiment

In [1]:
# encoding=utf8  
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
import os
import glob
import re
import datetime
import time
import codecs
import pandas as pd
%matplotlib inline
from textblob import TextBlob
from pattern import web 
from pattern.en.wordnet import sentiment
import pattern
from pattern.web import Twitter, plaintext
from pattern.db  import Datasheet
from pattern.en  import sentiment as sentiment_en
import re

Authorise Twitter Connection:

In [ ]:
from tweepy import OAuthHandler

# First we login into twitter

ckey = 'FjPzOxA6ZxHL4SJvNmU9ssLLt'
csecret = '5kYQPBYaJnLuk2ltbBxxDkWNTPvok9uoebtBLmgej1UtNVXMYd'
atoken = '844185356408946688-gXSe9Olnwjs9vDNjJW30fXbS9fWhww5'
asecret = 'wNGWGCXnWCRBKGiJKYjaTHgbqXihiE11TWc46QfaBAT3V'

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)  
api = tweepy.API(auth)

Use Twitter Search API to gather Tweets and then save to a csv file

In [ ]:
# Open/Create a file to append data
csvFile = open('Tweets_Harvest2.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)
max_tweets = 10000
for tweet in tweepy.Cursor(api.search, 
                    q="S&P500", 
                    since="2017-04-10", 
                    until="2017-04-14", 
                    lang="en").items(max_tweets):
    #Write a row to the csv file/ using utf-8
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    print tweet.created_at, tweet.text
csvFile.close()

Load in harvested tweets to Pandas dataframe.

In [27]:
import pandas as pd
import numpy as np
col_names = ['Date','Tweet']
data1 = pd.read_csv('Tweets_Harvest1.csv', names=col_names, index_col=False)
data2 = pd.read_csv('Tweets_Harvest2.csv', names=col_names, index_col=False)
data3 = pd.read_csv('Tweets_Harvestm.csv', names=[ 'Tweet', 'Date'], index_col=False)
data3 = data3.tail(1926)
data4 = pd.read_csv('Tweets_Harvest_LateMarch.csv', names=col_names, index_col=False)

data3

,Tweet,Date
1,#SP500 Index #ETF - #SPY : \r\nTesting Daily S...,21/03/2017 14:42
2,"All the three major #US indexes opened higher,...",21/03/2017 14:41
3,#SP500 And people think it is a good idea to c...,21/03/2017 14:38
4,#SP500 hit #Gap and 200 MA (green) on 1 Hour T...,21/03/2017 14:37
5,$ES_F SP500 getting closer to testing its 200d...,21/03/2017 14:34
6,This move down in the #SP500 looks pretty str...,21/03/2017 14:33
7,RT @michaelbatnick: The bull market is eight y...,21/03/2017 14:31
8,"Tues 3/21/17 10:30 AM ET\r\n#Dow $20,842 -63\...",21/03/2017 14:31
9,RT @michaelbatnick: The bull market is eight y...,21/03/2017 14:26
10,"RT @allstarcharts: No, This Is Not An 8-Year B...",21/03/2017 14:26


In [29]:
data1['Date'] = pd.to_datetime(data1['Date'], format="%d/%m/%y %H:%M")
data2['Date'] = pd.to_datetime(data2['Date'], format="%Y-%m-%d %H:%M:%S")
data3['Date'] = pd.to_datetime(data3['Date'], format="%d/%m/%Y %H:%M")
data4['Date'] = pd.to_datetime(data4['Date'], format="%Y-%m-%d %H:%M:%S")

data4

,Date,Tweet
0,2017-04-03 12:29:06,@HORANCapitalAdv Good news about the S&amp;P50...
1,2017-04-03 12:28:33,RT @1800ASENNA: S&amp;P500 minor TL breach. Th...
2,2017-04-03 12:27:27,Self-Driving Cars Raise Questions About Who Ca...
3,2017-04-03 12:19:18,#S&amp;P500..Is not looking that great to be f...
4,2017-04-03 12:14:27,"#ES eMini S&amp;P500 Futures Trading in range,..."
5,2017-04-03 12:02:54,Is the #S&amp;P500 getting ready to crash in #...
6,2017-04-03 11:53:15,Here's What Getting 20% Less Than You Deserve ...
7,2017-04-03 11:46:23,@takis2910 @henrictamm Does it look the same w...
8,2017-04-03 11:32:26,Holding on to S&amp;P500 short for 4th week – ...
9,2017-04-03 11:20:10,Sheryl Sandberg's https://t.co/KejhKR764X and ...


In [31]:
#combine datasets
frames = [data3, data4,data2, data1]
data = pd.concat(frames)

#remove null values
data['Tweet'].replace('', np.nan, inplace = True)
data.dropna(subset=['Tweet'], inplace= True)

In [32]:
#Remove some special characters in the Tweet column 
data['Tweet'] = data.Tweet.str.replace('\r\n','') #Remove the new-line character
data['Tweet'] = data.Tweet.str.replace('\r','') #Remove the tab character
data['Tweet'] = data.Tweet.str.replace('\n','') #Remove the tab character
data['Tweet'] = data.Tweet.str.replace('\,','') #Remove the single quote
data['Tweet'] = data.Tweet.str.replace('\,','') #Remove the comma
data['Tweet'] = data.Tweet.str.replace('\@','') #Remove the @ mark
data['Tweet'] = data.Tweet.str.replace('\"','') #Remove the double quote
data['Tweet'] = data.Tweet.str.replace('\t','') #Remove the tab character
data['Tweet'] = data.Tweet.str.replace('\#','') #Remove the tab character
data['Tweet'] = data.Tweet.str.replace('http\S+','') #Remove the tab character
#Step 3) Save the data to a new CSV file
data.to_csv('clean_tweet_new1.csv',index=False)

In [33]:
import numpy as np
from itertools import product


#col_names = ['Date','Tweet']
df = pd.read_csv('clean_tweet_new1.csv')

#df = df[pd.notnull(df['Tweet'])]
#df['Date'] = pd.to_datetime(df['Date'])

i = 0
length = len(df['Tweet'])
while i<length-2:
    j = length-1
    while j>i:
        if str(df['Tweet'][i])[:20] in str(df['Tweet'][j]):
            df['Tweet'][i] = np.nan
        j=j-1
    i=i+1

df.dropna(subset=['Tweet'], inplace= True)
 

In [39]:
    
def remove_emoji(data):
    if not data:
        return data
    if not isinstance(data, basestring):
        return data
    try:
        patt = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    except re.error:
        patt = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')
    return patt.sub('', data)

df

IndentationError: expected an indented block (<ipython-input-39-342efc0b35dd>, line 3)

In [40]:
df.to_csv('clean_tweet_removed_duplicates.csv', index=False)

In [ ]:
i=0
threshold=0
pos_sent_tweet=0
neg_sent_tweet=0

for tweet in df['Tweet']:
    if tweet.isalnum:
        print tweet
        s = plaintext(tweet)
        w = sentiment_en(s)
        i=i+1
        print(i)
        print(tweet)
        print(w)

In [44]:
range(3,5)

[3, 4]

In [46]:
import re
from __future__ import division

df = pd.read_csv('clean_tweet_removed_duplicates.csv')


i=0

dates = []
pos_sentiment = []
neg_sentiment = []
neut_sentiment = []
Number_of_tweets = []


threshold=0
for k in range(3,5):
    for j in range(1,32):

        pos_sent_tweet=0
        neg_sent_tweet=0
        null_pol = 0
        print(j)
        if j < 9: 
            start_date = ('2017-0%s-0%s'%(k, j))
            end_date = ('2017-0%s-0%s'%(k,(j+1)))
        elif j == 9 :
            start_date = ('2017-0%s-0%s'%(k, j))
            end_date = ('2017-0%s-%s'%(k,(j+1)))
        else:
            start_date = ('2017-0%s-%s'%(k, j))
            end_date = ('2017-0%s-%s'%(k,(j+1)))

        dates.append(start_date)
        mask = (df['Date'] >= start_date) & (df['Date'] < end_date)
        df1 = df.loc[mask]
        Number_of_tweets.append(len(df1['Tweet']))
        Number_of_tweets_in = len(df1['Tweet'])
        for tweet in df1['Tweet']:
            if tweet.isalnum:
    #             tweet = remove_emoji(tweet).replace("\\", "\\\\")
                tweet = re.sub('[^a-zA-Z0-9_*. ]','',tweet)
                analysis = TextBlob(tweet)
        #         print(i)
        #         i = i+1
        #         print(tweet)
        #         print(analysis.sentiment.polarity)
                if analysis.sentiment.polarity>threshold:
                    pos_sent_tweet=pos_sent_tweet+1
                elif analysis.sentiment.polarity<threshold:
                    neg_sent_tweet=neg_sent_tweet+1
                else:
                    null_pol = null_pol + 1
        print('The tweets collected between: %s and %s'%(start_date, end_date))
        print('Number of positive tweets are: %d' % pos_sent_tweet)
        print('Number of negative tweets are: %d' % neg_sent_tweet)
        print('Number of neutral tweets are: %d' % null_pol)
        if Number_of_tweets_in == 0:
            pos_sentiment.append(0)
            neg_sentiment.append(0)
            neut_sentiment.append(0)
        else:
            pos_sentiment.append(pos_sent_tweet/Number_of_tweets_in)
            neg_sentiment.append(neg_sent_tweet/Number_of_tweets_in)
            neut_sentiment.append(null_pol/Number_of_tweets_in)

df

,Date,Tweet
0,2017-03-21 14:41:00,All the three major US indexes opened higher w...
1,2017-03-21 14:38:00,SP500 And people think it is a good idea to ch...
2,2017-03-21 14:37:00,SP500 hit Gap and 200 MA (green) on 1 Hour Tim...
3,2017-03-21 14:34:00,$ES_F SP500 getting closer to testing its 200d...
4,2017-03-21 14:33:00,This move down in the SP500 looks pretty stru...
5,2017-03-21 14:31:00,Tues 3/21/17 10:30 AM ETDow $20842 -63SP500 $...
6,2017-03-21 14:24:00,SP500 close short and pull back one more time ...
7,2017-03-21 14:15:00,$ES SP500 ES_F under $2370 ok go to $2340 sell...
8,2017-03-21 14:12:00,Nordstrom (NYSE:JWN) loses -3.05% on Monday-To...
9,2017-03-21 14:05:00,SP500 nice algos gap thanks :)


In [47]:
df3 = pd.DataFrame({'Date': dates, 
                    'Pos_T_Sentiment': pos_sentiment, 
                    'Neg_T_Sentiment': neg_sentiment, 
                    'Neut_T_Sentiment': neut_sentiment, 
                    'Number_of_Tweets': Number_of_tweets})

df3.to_csv("Twitter_sentiment2.csv", index=False)

df3

,Date,Neg_T_Sentiment,Neut_T_Sentiment,Number_of_Tweets,Pos_T_Sentiment
0,2017-03-01,0.000000,0.000000,0,0.000000
1,2017-03-02,0.000000,0.000000,0,0.000000
2,2017-03-03,0.000000,0.000000,0,0.000000
3,2017-03-04,0.000000,0.000000,0,0.000000
4,2017-03-05,0.000000,0.000000,0,0.000000
5,2017-03-06,0.000000,0.000000,0,0.000000
6,2017-03-07,0.000000,0.000000,0,0.000000
7,2017-03-08,0.000000,0.000000,0,0.000000
8,2017-03-09,0.000000,0.000000,0,0.000000
9,2017-03-10,0.000000,0.000000,0,0.000000


# Financial Prediction

In [ ]:
# load the dataset
df = pd.read_csv('SP500.csv',engine='python')

dates = df.index.values
prices = df['Close'].values

print dates 
print prices
#dataset = df.values
#dataset = dataset.astype('float32')
#plt.plot(dataset)

In [ ]:
def create_datasets(dates,prices):
    train_size=int(0.80*len(dates))
    TrainX,TrainY=[],[]
    TestX,TestY=[],[]
    cntr=0
    for date in dates:
        if cntr<train_size:
            TrainX.append(date)
        else:
            TestX.append(date)    
    for price in prices:
        if cntr<train_size:
            TrainY.append(price)
        else:
            TestY.append(price)
            
    return TrainX,TrainY,TestX,TestY

def predict_prices(dates,prices,x):
    TrainX,TrainY,TestX,TestY=create_datasets(dates,prices)

    TrainX=np.reshape(TrainX,(len(TrainX),1))
    TrainY=np.reshape(TrainY,(len(TrainY),1))
    TestX=np.reshape(TestX,(len(TestX),1))
    TestY=np.reshape(TestY,(len(TestY),1))
    
    #for i in range(251):
     #   print TrainX[i],TrainY[i],'\n'
    
    
    model=Sequential()
    model.add(Dense(32,input_dim=1,init='uniform',activation='relu'))
    model.add(Dense(32,input_dim=1,init='uniform',activation='relu'))
    model.add(Dense(16,init='uniform',activation='relu'))
    
    model.add(Dense(1,init='uniform',activation='relu'))
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
    model.fit(TrainX,TrainY,nb_epoch=100,batch_size=3,verbose=1)

In [ ]:
predict_prices(dates,prices,2)